## Домашнее задание 10 ##
### Это задание выполнено Павлом Овчинниковым при обучении на курсе "Теория вероятностей и математическая статистика" ###
### Преподаватель Дамир Тлеубаев ###

### Задание 1)
Провести дисперсионный анализ для определения того, есть ли различия среднего роста среди взрослых футболистов, хоккеистов и штангистов. Даны значения роста в трех группах случайно выбранных спортсменов: Футболисты: 173, 175, 180, 178, 177, 185, 183, 182. Хоккеисты: 177, 179, 180, 188, 177, 172, 171, 184, 180. Штангисты: 172, 173, 169, 177, 166, 180, 178, 177, 172, 166, 170. 
### Решение: 
Так как на 1 количественный показатель - рост спортсмена, влияет 1 качественный критерий (фактор) - вид спорта, то применим однофакторной дисперсионной анализ. Проверяем на условия применимости дисперсионного анализа:
- соблюдается независимость измерений, так как это разные виды спорта
- соблюдается нормальность распределения (проверим тестом Шапиро-Уилка)
- выборки имеют разный размер, поэтому неоднородность может иметь сильное влияние на результат. Дисперсия выборок $ s^2 = Σ (x_i – x )^2 / (n-1) $

In [13]:
import pandas as pd
import numpy as np
import scipy.stats as stats
from scipy import stats
from scipy.stats import shapiro
from scipy.stats import bartlett
import matplotlib.pyplot as plt

ftb=np.array([173, 175, 180, 178, 177, 185, 183, 182]) # footballer
hk=np.array([177, 179, 180, 188, 177, 172, 171, 184, 180]) # hockey 
wl=np.array([172, 173, 169, 177, 166, 180, 178, 177, 172, 166, 170]) # Weightlifter 
sportsmens=np.array(['Футболисты', 'Хоккеисты', 'Штангисты']) #  

# # средние значения
ftb_avg = np.mean(ftb)  
hk_avg = np.mean(hk)
wl_avg = np.mean(wl)

print(f'средние значения выборок "Футболисты", "Хоккеисты", "Штангисты" соответственно {ftb_avg}, {hk_avg}, {wl_avg}')

# тест Шапиро-Уилка
def test_shapiro(data):
    stat, p_value = shapiro(data)
    alpha = 0.05
    if p_value < alpha:
        print('Данные {data} НЕ имеют нормального распределения.')
    else:
        print(f'Данные {data} нормального распределения.')

test_shapiro(ftb)
test_shapiro(hk)
test_shapiro(wl)

print (stats.bartlett(ftb, hk, wl))

print(f'Дисперсия выборок "Футболисты", "Хоккеисты", "Штангисты" соответственно {np.var(ftb)}, {np.var(hk)}, {np.var(wl)}')

средние значения выборок "Футболисты", "Хоккеисты", "Штангисты" соответственно 179.125, 178.66666666666666, 172.72727272727272
Данные [173 175 180 178 177 185 183 182] нормального распределения.
Данные [177 179 180 188 177 172 171 184 180] нормального распределения.
Данные [172 173 169 177 166 180 178 177 172 166 170] нормального распределения.
BartlettResult(statistic=0.4640521043406442, pvalue=0.7929254656083131)
Дисперсия выборок "Футболисты", "Хоккеисты", "Штангисты" соответственно 14.859375, 25.33333333333333, 20.925619834710744


По условию сравниваем 3 группы, значит возможно применение критерия Фишера
$$ F_н = \frac {\sigma_{факт}^2}{\sigma_{ост}^2} $$
, где $\sigma_{факт}^2 = \frac {S_ф^2}{(k-1)}$  и $\sigma_{ост}^2 = \frac {S_{ост}^2}{(n-k)} $
$$ s_ф^2 = \sum_{i=1}^k (\overline {y_i} - \overline {Y})^2*n_i $$

Применим метод однофакторного дисперсионного анализа из библиотеки scipy:



In [14]:
k=3 # количество сравниваемых выборок
k1 = 2 # степень свободы 3-1
n=len(ftb)+len(hk)+len(wl) # общее кол-во измерений во всех выборках
k2 = n-k
print(k, n, k1, k2)
Fk = 3.38

stats.f_oneway(ftb, hk, wl)

3 28 2 25


F_onewayResult(statistic=5.500053450812596, pvalue=0.010482206918698693)

Таким образом нашли, Fн = 5.5, табличное Fk = 3.38 при уровне значимости  alpha = 5% и p_value = 0.01. Fн > Fk, и мы нашли статистически значимые отличия.

Провести дополнительную проверку по методу Post hoc test Tukey не представляется возможным, т.к. все массивы выборок должны быть одинаковой длины.

Но для общего понимания, проведен расчет по наборам данных, где недостающие испытания дополнены средними значения по выборке )))
Получим следующий результат:

In [15]:
from statsmodels.stats.multicomp import pairwise_tukeyhsd

df=pd.DataFrame({"score": [ 173, 175, 180, 178, 177, 185, 183, 182, 179.125, 179.125, 179.125,
                          177, 179, 180, 188, 177, 172, 171, 184, 180, 178.667, 178.667,
                          172, 173, 169, 177, 166, 180, 178, 177, 172, 166, 170],
 "group":np.repeat(["Футболисты","Хоккеисты","Штангисты"], repeats =11)})

tukey=pairwise_tukeyhsd(df["score"],  df["group"],  alpha =0.05)
print(tukey)

    Multiple Comparison of Means - Tukey HSD, FWER=0.05     
  group1     group2  meandiff p-adj   lower    upper  reject
------------------------------------------------------------
Футболисты Хоккеисты  -0.4583 0.9675  -5.0686  4.1521  False
Футболисты Штангисты  -6.3977  0.005 -11.0081 -1.7874   True
 Хоккеисты Штангисты  -5.9395 0.0094 -10.5498 -1.3291   True
------------------------------------------------------------


Различий между Футболистами Хоккеистами не найдено p_value = 0.9675 False
Найдены различия между: 
Футболистами Штангистами  p_value= 0.005 True
Хоккеистами Штангистами  p_value= 0.0094 True